In [1]:
import numpy as np
import os, sys, pickle
import time
import nest
import random
import quantities as pq  # noqa
from elephant.spike_train_generation import StationaryPoissonProcess


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.6.0
 Built: Sep 28 2023 11:53:34

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



In [2]:
def pITD_unimodal(N, mu1=0, sigma1=10):
    return np.random.normal(mu1, sigma1, int(N))

    return np.array(X)

def pITD_bimodal(N, mu1=-160, mu2=160, sigma1=0, sigma2=0):
    X1 = np.random.normal(mu1, sigma1, int(np.floor(N/2)))
    X2 = np.random.normal(mu2, sigma2, int(np.ceil(N/2)))
    X = np.concatenate([X1, X2])
    np.random.shuffle(X)
    return np.array(X)

def ITD_to_rad(x, mITD=160):
    return (x+mITD)/((2*mITD)/np.pi)

def generate_poisson(lamb, t_stop=1150, t_start=150):
    spiketrain_array = StationaryPoissonProcess(rate=lamb*pq.Hz,t_stop=t_stop*pq.ms,t_start=t_start*pq.ms).generate_spiketrain(as_array=True)
    return spiketrain_array

In [3]:
gen_po = 1

# comment out to have a different randomization of random parameters each time
#np.random.seed(1234)

# total number of neurons in the network
N = 1000
# fraction of excitatory neurons
exc_inh = 0.8
# number of excitatory neurons
NE = int(exc_inh *N)
# number of inhibitory neurons
NI = N - NE

n_smpl = 0

# probability of (E --> E,I) connections
eps_exc = 0.2
# probability of (I --> E,I) connections
eps_inh = 0.5

# inhibition dominance of synaptic weights (IPSP = g x EPSP)
g = 8#10.
# EPSP strength (mV) of recurrent synapses in the local network
J_rec = .2
# (E,I) to (E,I) weights (mV)
J_exc = J_ee = J_ei = J_rec
J_inh = J_ie = J_ii = -g*J_rec

# number of neurons in the non-local network
n_ext = 1000.
# spontaneuos firing rate of the non-local network
r_ext = 1.
# overall rate of the non-local input
s_ext = n_ext * r_ext
# EPSP strength (mV) of synapses from non-local sources
J_ext = .2

# number of LGN neurons contacting one cortical neuron
n_ffw = 50.
# average firing rate of LGN neurons
r_ffw = 20.
# overall rate of the feedforward input
r_base = n_ffw * r_ffw
# EPSP strength (mV) of feedforward synapses
J_ffw = 1.

# average synaptic delay (ms)
delay = 1.
# modulation in the input
input_mod = 0.1#np.array(NE*[0.2] + NI*[.02])

# initial random preferred orientation (PO) of neurons
if gen_po:
    np.random.seed(1234)
    po_init = pITD_bimodal(N)
    file = open('po_init', 'wb'); pickle.dump(po_init, file); file.close()
    #np.random.uniform(-160., 160, N)
#po_init[0:n_smpl] = np.arange(0, 180, 180/12.)*np.pi/180
#po_init[NE:NE+n_smpl] = np.arange(0, 180, 180/12.)*np.pi/180

else:
    f = open('po_init', 'rb'); po_init = pickle.load(f); f.close()

# number of stimulus orientations to show
stim_no = 11
# range of stimuli
stim_range = np.arange(-160., 160+1, 2*160/(stim_no-1))
# trial
trial = 1
# simulation time
simtime = 1*1000.
# transient time at the beginning of the simulation
t_trans = 150.

# membrane time constant (ms)
tauMem = 20.
# voltage threshold (mV)
theta = 20.

n_cores = 4
# neuron parameters (current-based with delta synapses)
neuron_params_delta = {"C_m"      : 1.0,
                "tau_m"     : tauMem,
                "t_ref"     : 2.0,
                "E_L"       : 0.0,
                "V_min"     : -np.inf,
                "V_m"       : 0.,
                "V_reset"   : 0.,
                "V_th"      : theta,
                "I_e"       : 0.}

#########################

# define the NEST path here if needed
#nest_path = '/Users/sadra/NEST/nest/ins/lib/python3.4/site-packages/'
#if os.path.exists(nest_path): sys.path.append(nest_path)

# code path
code_path = os.getcwd()

# results path
res_path = code_path+'/Results/'
if not os.path.exists(res_path): os.mkdir(res_path)

#########################
#########################

In [4]:
def NEST_initiate(dt=.1, n_cores=1): # Devemos excluir essa parte?
    '''
    Inicializando o NEST

    dt: resolucao das simulacoes
    n_cores: quantos cores devem ser utilizados para a simulacao
    '''

    #NEST restart
    nest.ResetKernel()
    rng_seed = int(np.random.uniform(0, 9876)) #int(np.random.uniform(0, 12345))
    nest.SetKernelStatus({'rng_seed': rng_seed, #Use GetStatus to see all options
                          'resolution': dt,
                          'print_time': True,
                          'overwrite_files': True,
                          'local_num_threads': n_cores})



In [5]:
# --- stimulating the network in response to one stimulus orientation
def stimulate_network(stim_deg, stim_id=str(0)):

    # --- Start
    ti = time.time()

    NEST_initiate(n_cores=n_cores)

    print("Network ...")

    # spiking neurons
    nest.SetDefaults("iaf_psc_delta", neuron_params_delta)
    nodes_ex = nest.Create("iaf_psc_delta", NE)
    nodes_in = nest.Create("iaf_psc_delta", NI)

    nodes_all = nodes_ex + nodes_in

    # vector of total input to all neurons
    input_rate = r_base *(1.+ input_mod* np.cos(1*(ITD_to_rad(stim_deg) - ITD_to_rad(po_init)) ))

    # -- define synapses
    #external
    nest.CopyModel("static_synapse", "ext", {"weight":J_ext, "delay":delay})
    #feedforward
    nest.CopyModel("static_synapse", "ffw", {"weight":J_ffw, "delay":delay})
    # E-to-E
    nest.CopyModel("static_synapse", "exc", {"weight":J_exc, "delay":delay})
    # I-to-I
    nest.CopyModel("static_synapse", "inh", {"weight":J_inh, "delay":delay})

    # -- SETTING DEVICES TO RECORD

    # spike detector
    sp_all = nest.Create("spike_recorder", 1)

    # stationary
    nest.SetStatus(sp_all[0],
                   {"label":"spikes-all-"+stim_id+'-tr'+str(trial),
                    'record_to': "ascii",
                    "start": 0 + t_trans, "stop": simtime })

    nest.Connect(nodes_all, sp_all[0],  syn_spec={'synapse_model': 'ext'})

    # -- Connecting the network
    print("Connecting the network ...")
    
    CE = int(eps_exc* NE)
    CI = int(eps_inh* NI)

    ffw_inp = sum(nest.Create('spike_train_injector', params={"spike_times": generate_poisson(rate), "allow_offgrid_times":True}) for rate in input_rate)
    nest.Connect(ffw_inp, nodes_all, "one_to_one")

    ext_inp = nest.Create("poisson_generator")
    nest.SetStatus(ext_inp, {"rate":s_ext})
    nest.Connect(ext_inp, nodes_all, syn_spec={'synapse_model': "ext"})


    ### recurrent connectivity
    CE = int(eps_exc* NE)
    CI = int(eps_inh* NI)

    nest.rng_seed = 12345

    nest.Connect(nodes_ex, nodes_all, {"rule": "fixed_indegree", "indegree": CE, 'allow_autapses': False, "allow_multapses": False},syn_spec={'synapse_model': "exc"})
    nest.Connect(nodes_in, nodes_all, {"rule": "fixed_indegree", "indegree": CI, 'allow_autapses': False, "allow_multapses": False},syn_spec={'synapse_model': "inh"})
    
    # --- running simulations

    print("Simulating the network ...")


    print('# -- Trial # ', str(trial))
    nest.Simulate(simtime + t_trans)

    ts = time.time()
    sim_time = ts - ti
    r_avg = nest.GetStatus(sp_all[0], 'n_events')[0]/ (N*simtime/1000.)

    print('\n########################################')
    print('End of simulation.')
    print("Simulation time   : %.2f s" % sim_time)
    print("Average rate      : %.2f Hz" %  r_avg)
    print('######################################## \n')

In [6]:
def run_simulation():
    '''
    Simulates the response of a network of spiking neurons wit a defined network connectivity
    in response to different stimulus orientations.
    '''

    # folder
    sim_folder = 'N'+str(N)+ 'g'+str(g)

    if sim_folder in os.listdir(res_path):
        os.chdir(res_path+sim_folder)
    else:
        os.mkdir(res_path+sim_folder)
        os.chdir(res_path+sim_folder)

    # stimulation
    for stim in enumerate(stim_range):
        print('# ---')
        print('# -- orientation: ', \
        str(stim[0]+1),'/',str(stim_no)+' ('+str(stim[1])+' ms)')
        print('# ---')

        stim_id = 'st'+str(stim[0])
        stimulate_network(stim_deg=stim[1], stim_id = stim_id)

    os.chdir(code_path)

In [7]:
trial_no = 10

In [8]:
# --- running network simulations for different degrees
trial = 0
con_gen = 0

run_simulation()
con_gen = 0

while trial < trial_no:
    trial = trial + 1
    run_simulation()


# ---
# -- orientation:  1 / 11 (-160.0 ms)
# ---

Jan 23 09:38:09 SimulationManager::set_status [Info]: 
Network ...
    Temporal resolution changed from 0.1 to 0.1 ms.
Connecting the network ...
Simulating the network ...
# -- Trial #  0

########################################
End of simulation.
Simulation time   : 7.49 s
Average rate      : 3.99 Hz
######################################## 

# ---
# -- orientation:  2 / 11 (-128.0 ms)
# ---

Jan 23 09:38:16 NodeManager::prepare_nodes [Info]: 
Network ...
    Preparing 2008 nodes for simulation.

Jan 23 09:38:16 SimulationManager::start_updating_ [Info]: 
    Number of local nodes: 2008
    Simulation time (ms): 1150
    Number of OpenMP threads: 4
    Not using MPI

[   2% ] Model time: 14Connecting the network ....4065
[ 100% ] Model time: 1150.0 ms, Real-time factor: 0.6339

Jan 23 09:38:16 SimulationManager::run [Info]: 
    Simulation finished.

Jan 23 09:38:16 SimulationManager::set_status [Info]: 
    Temporal resolution chan

Simulating the network ...
# -- Trial #  48

Dec 17 20:05:07 NodeManager::prepare_nodes [Info]: 
    Preparing 2003 nodes for simulation.

Dec 17 20:05:07 SimulationManager::start_updating_ [Info]: 
    Number of local nodes: 2003
    Simulation time (ms): 1150
    Number of OpenMP threads: 1
    Not using MPI

[   5% ] Model time: 51.0 ms, Real-time factor: 1.5045
########################################
End of simulation.
Simulation time   : 3.03 s
Average rate      : 2.52 Hz
######################################## 

# ---
# -- orientation:  30 / 51 (25.600000000000165 ms)
# ---
Network ...
[  11% ] Model time: 123.0 ms, Real-time factoConnecting the network ...
Connecting the network ...
[ 100% ] Model time: 1150.0 ms, Real-time factor: 1.7188

Dec 17 20:05:09 SimulationManager::run [Info]: 
    Simulation finished.

Dec 17 20:05:09 SimulationManager::set_status [Info]: 
    Temporal resolution changed from 0.1 to 0.1 ms.
Simulating the network ...
# -- Trial #  48

###############

[ 100% ] Model time: 1150.0 ms, Real-time factor: 1.6554

Dec 17 20:05:36 SimulationManager::run [Info]: 
    Simulation finished.

Dec 17 20:05:36 SimulationManager::set_status [Info]: 
    Temporal resolution changed from 0.1 to 0.1 ms.


Simulating the network ...
# -- Trial #  48

Dec 17 20:05:37 NodeManager::prepare_nodes [Info]: 
    Preparing 2003 nodes for simulation.

########################################
End of simulation.
Simulation time   : 2.97 s
Average rate      : 2.50 Hz
######################################## 

# ---
# -- orientation:  41 / 51 (96.00000000000023 ms)
# ---

Dec 17 20:05:37 SimulationManager::start_updating_ [Info]: 
    Number of local nodes: 2003
    Simulation time (ms): 1150
    Number of OpenMP threads: 1
    Not using MPI

[   2% ] Network ... 14.0 ms, Real-time factor: 1.3990
[   5% ] Model time: 51.0 ms,Connecting the network ...
Connecting the network ...
[ 100% ] Model time: 1150.0 ms, Real-time factor: 1.6281

Dec 17 20:05:39 SimulationManager::run [Info]: 
    Simulation finished.

Dec 17 20:05:39 SimulationManager::set_status [Info]: 
    Temporal resolution changed from 0.1 to 0.1 ms.
Simulating the network ...
# -- Trial #  48

Dec 17 20:05:40 NodeManager::prepare_nodes [

[ 100% ] Model time: 1150.0 ms, Real-time factor: 1.5826

Dec 17 20:06:07 SimulationManager::run [Info]: 
    Simulation finished.

Dec 17 20:06:07 SimulationManager::set_status [Info]: 
    Temporal resolution changed from 0.1 to 0.1 ms.


...

[ 100% ] Model time: 1150.0 ms, Real-time factor: 1.3679

Dec 17 20:09:56 SimulationManager::run [Info]: 
    Simulation finished.

Dec 17 20:09:56 SimulationManager::set_status [Info]: 
    Temporal resolution changed from 0.1 to 0.1 ms.
Simulating the network ...
# -- Trial #  50

Dec 17 20:09:57 NodeManager::prepare_nodes [Info]: 
    Preparing 2003 nodes for simulation.

Dec 17 20:09:57 SimulationManager::start_updating_ [Info]: 
    Number of local nodes: 2003
    Simulation time (ms): 1150
    Number of OpenMP threads: 1
    Not using MPI

[   2% ] Model time: 14.0 ms, Real-time factor: 1.4587
########################################
End of simulation.
Simulation time   : 3.01 s
Average rate      : 2.45 Hz
######################################## 

# ---
# -- orientation:  32 / 51 (38.400000000000176 ms)
# ---
[   3% ] Model timeNetwork ...Real-time factor: 1.5458
[  10% ] Model time: 105.0 ms, Real-time factor: 1.752Connecting the network ...
Connecting the network ...
[ 100% ] 

[  89% ] Model time: 1

[ 100% ] Model time: 1150.0 ms, Real-time factor: 1.5732

Dec 17 20:10:22 SimulationManager::run [Info]: 
    Simulation finished.

Dec 17 20:10:22 SimulationManager::set_status [Info]: 
    Temporal resolution changed from 0.1 to 0.1 ms.
Simulating the network ...
# -- Trial #  50

########################################
End of simulation.
Simulation time   : 2.63 s
Average rate      : 2.52 Hz
######################################## 

# ---
# -- orientation:  41 / 51 (96.00000000000023 ms)
# ---

Dec 17 20:10:23 NodeManager::prepare_nodes [Info]: 
    Preparing 2003 nodes for simulation.

Dec 17 20:10:23 SimulationManager::start_updating_ [Info]: 
    Number of local nodes: 2003
    Simulation time (ms): 1150
    Number of OpenMP threads: 1
    Not using MPI

[   3% ] Model timeNetwork ...Real-time factor: 1.2397
[  33% ] Model time: 373.0 ms, Real-time factoConnecting the network ...
Connecting the network ...
[ 100% ] Model time: 1150.0 ms, Real-time factor: 1.3874] Model time: 58

[ 100% ] Mo

[ 100% ] Model time: 1150.0 ms, Real-time factor: 1.5592

Dec 17 20:10:46 SimulationManager::run [Info]: 
    Simulation finished.

Dec 17 20:10:46 SimulationManager::set_status [Info]: 
    Temporal resolution changed from 0.1 to 0.1 ms.
Simulating the network ...
# -- Trial #  50

Dec 17 20:10:47 NodeManager::prepare_nodes [Info]: 
    Preparing 2003 nodes for simulation.

########################################
End of simulation.
Simulation time   : 2.65 s
Average rate      : 2.53 Hz
######################################## 

# ---
# -- orientation:  50 / 51 (153.60000000000025 ms)
# ---
Network ...

Dec 17 20:10:47 SimulationManager::start_updating_ [Info]: 
    Number of local nodes: 2003
    Simulation time (ms): 1150
    Number of OpenMP threads: 1
    Not using MPI

[   6% ] Model time: 69.0 ms, Real-timeConnecting the network ...
Connecting the network ...
[ 100% ] Model time: 1150.0 ms, Real-time factor: 1.3943: 1.4075 ] Model time: 713.0 ms, Real-time factor: 1.4128 ] Model